# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [3]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [8]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']

data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [9]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [26]:
model = Word2Vec(data,
                 size=100,
                 window=5,
                 min_count=1,
                 workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [27]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563952, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [28]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [29]:
wv.most_similar('Texas')

[('Louisiana', 0.8216902613639832),
 ('Pennsylvania', 0.8205119371414185),
 ('Connecticut', 0.8110950589179993),
 ('California', 0.7976654767990112),
 ('Oklahoma', 0.7945061326026917),
 ('Georgia', 0.7921913862228394),
 ('Illinois', 0.7890644073486328),
 ('Arkansas', 0.7872691750526428),
 ('Maryland', 0.787248969078064),
 ('Florida', 0.7850942611694336)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [30]:
wv.most_similar(negative='Texas')

[('much-vaunted', 0.40408897399902344),
 ('Parent/Grandparent', 0.3983173370361328),
 ('Animal-rescue', 0.39527833461761475),
 ('Burned-Out', 0.38667669892311096),
 ('Optimizing', 0.38215315341949463),
 ('Headstrong', 0.3777387738227844),
 ('once-reliable', 0.37068381905555725),
 ('Uglies', 0.36835429072380066),
 ('Unabashedly', 0.3633638322353363),
 ('BRFWA', 0.36264294385910034)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [31]:
wv['Texas']

array([-1.4098985 ,  0.75261265,  0.7151542 ,  1.959742  ,  1.6929795 ,
       -1.8076197 ,  0.18581842, -0.68432575, -1.8373393 , -0.8437005 ,
       -0.94088197, -0.26523966, -1.9357558 , -2.4865332 ,  2.1262553 ,
        1.5207037 , -0.55991966,  1.4821209 , -0.30073047, -0.10704461,
        0.7250444 ,  1.2302105 ,  0.26104686,  0.10348058,  0.55814725,
        0.62871116,  1.4713173 , -0.7074577 , -0.2748879 , -3.563481  ,
        0.36350265,  1.7945241 , -1.3287064 ,  0.55829334,  1.283131  ,
        1.3238305 , -0.54689693, -2.641253  ,  2.2877665 ,  1.7375817 ,
       -1.2855299 ,  1.2651082 , -0.7101259 , -0.25037313,  0.7827085 ,
        0.70119977,  0.3254168 ,  0.72281057, -0.61419773, -0.54588866,
        0.6998707 ,  1.300283  , -0.020726  ,  0.66383195, -1.1357073 ,
        1.6229748 ,  0.19525991,  0.24543378, -0.8640934 ,  2.4620283 ,
        1.6340982 , -1.9069481 , -0.60668695,  2.7393804 ,  3.5821428 ,
        0.60670084, -0.6922799 ,  1.3180945 , -0.848681  , -0.46

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [32]:
wv.vectors

array([[ 1.0316389e+00,  1.8104901e+00,  6.2194634e-02, ...,
        -1.1046988e+00,  1.4356217e+00,  1.3531760e+00],
       [ 1.3405310e+00,  1.2696702e+00, -8.3731085e-02, ...,
        -2.1348543e+00,  9.3409044e-01,  2.0680850e+00],
       [ 1.8231044e+00, -1.4078567e+00, -9.2039084e-01, ...,
        -1.1058701e+00, -3.6644274e-01,  1.0363896e+00],
       ...,
       [-6.8368390e-02,  2.0048695e-03, -1.4707014e-01, ...,
        -5.3764079e-02, -1.0350694e-02,  4.8693612e-02],
       [-4.3386858e-02, -3.6307830e-02, -3.3416830e-02, ...,
        -4.3827970e-02,  9.7935367e-03, -1.8572982e-02],
       [-1.0073126e-03,  2.9725030e-02, -8.1388645e-02, ...,
         3.4563847e-02, -4.2839539e-03, -3.7427101e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [33]:
wv.most_similar(positive=['king','woman'], negative=['man'])

[('princess', 0.6094034910202026),
 ('title', 0.5895537734031677),
 ('villain', 0.5883733034133911),
 ('symbol', 0.57537841796875),
 ('monster', 0.5725193619728088),
 ('lover', 0.5708041787147522),
 ('Queen', 0.5623399019241333),
 ('brunette', 0.5606585144996643),
 ('revival', 0.5562289953231812),
 ('jewel', 0.553235650062561)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!